# Extrai tweets

In [ ]:
!pip install tweepy datetime

## Play 

In [ ]:
!python get_tweets.py

## Entendendo a estrutura do JSON

In [ ]:
import json

In [ ]:
## Abrir o arquivo de tweets e ler as linhas
with open('collected_tweet_2021-03-08-09-55-24.txt','r') as file:
    tweets = file.readlines()

In [ ]:
## Fazendo o parse do tweet para um formato json e exporta aquivo
with open('tweet.json','w') as f:
    json.dump(
         json.loads(json.loads(tweets[0])),
         f
    )

In [ ]:
 json.loads(
    json.loads(tweets[1])
        )

In [ ]:
parsed_tweets = [json.loads(json.loads(i)) for i in tweets]

In [ ]:
## Quantidade de tweets coletador
len(parsed_tweets)

## Acessando as chaves do JSON (Dicionário)

In [ ]:
primeirotweet = parsed_tweets[1]

In [ ]:
list(primeirotweet.items())[1]

In [ ]:
##Acessa o texto do tweet
#Chave de primeiro nível (root)
primeirotweet['text']

In [ ]:
##Acessar a chave USER
primeirotweet['user']

In [ ]:
## Acessando o screem name do user
primeirotweet['user']['screen_name']

In [ ]:
## Vamos pegar a informação do tweet original
primeirotweet['retweeted_status']['created_at']

In [ ]:
## Pegando o name do primeiro usuário mencionado no tweet original
primeirotweet['retweeted_status']['entities']['user_mentions']

# Transformar os tweets em uma estrutura relacional analisável

In [ ]:
import pandas as pd

In [ ]:
teste = pd.DataFrame(primeirotweet)
teste
#NÃO FUNCIONA !

In [ ]:
df_tratado = pd.DataFrame(primeirotweet).reset_index(drop=True).iloc[:1]
df_tratado

In [ ]:
df_tratado.columns

In [ ]:
df_tratado.drop(columns=['quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'user', 'entities', 'retweeted_status'], inplace=True)

In [ ]:
df_tratado.columns

In [ ]:
primeirotweet['user']

In [ ]:
df_tratado['user_id'] = primeirotweet['user']['id']
df_tratado['user_id_str'] = primeirotweet['user']['id_str']
df_tratado['user_screen_name'] = primeirotweet['user']['screen_name']
df_tratado['user_location'] = primeirotweet['user']['location']
df_tratado['user_description'] = primeirotweet['user']['description']
df_tratado['user_protected'] = primeirotweet['user']['protected']
df_tratado['user_verified'] = primeirotweet['user']['verified'] 
df_tratado['user_followers_count'] = primeirotweet['user']['followers_count'] 
df_tratado['user_friends_count'] = primeirotweet['user']['friends_count']
df_tratado['user_created_at'] = primeirotweet['user']['created_at'] 


In [ ]:
df_tratado

In [ ]:
#Entidades
primeirotweet['entities']

In [ ]:
user_mentions = []

for i in range(len(primeirotweet['entities']['user_mentions'])):
    dicionariobase = primeirotweet ['entities']['user_mentions'][i].copy()
    dicionariobase.pop('indices', None)
    df = pd.DataFrame(dicionariobase, index= [0])
    df = df.rename(columns={
        'screen_name' : 'entities_screen_name',
        'name' : 'entities_name',
        'id' : 'entities_id',
        'id_str' : 'entities_id_str'
    })
    user_mentions.append(df)

In [ ]:
user_mentions[0]

In [ ]:
dfs = []
for i in user_mentions:
    dfs.append(
        pd.concat([df_tratado.copy(), i], axis=0)
    )

In [ ]:
pd.concat(dfs, ignore_index=True)

Vamos criar uma função que faz todo o tratamento do tweet e cpnverte para um DataFrame pandas

In [ ]:
def tweet_para_df(tweet):
    try:
        df_tratado = pd.DataFrame(tweet).reset_index(drop = True).iloc[:1]
        df_tratado.drop(columns=['quote_count', 'reply_count', 'retweet_count', 'favorite_count', 
                             'favorited', 'retweeted', 'user', 'entities', 'retweeted_status'
                            ], inplace=True)
        df_tratado['user_id'] = tweet['user']['id']
        df_tratado['user_id_str'] = tweet['user']['id_str']
        df_tratado['user_screen_name'] = tweet['user']['screen_name']
        df_tratado['user_location'] = tweet['user']['location']
        df_tratado['user_description'] = tweet['user']['description']
        df_tratado['user_protected'] = tweet['user']['protected']
        df_tratado['user_verified'] = tweet['user']['verified'] 
        df_tratado['user_followers_count'] = tweet['user']['followers_count'] 
        df_tratado['user_friends_count'] = tweet['user']['friends_count']
        df_tratado['user_created_at'] = tweet['user']['created_at'] 

        user_mentions = []

        for i in range(len(tweet['entities']['user_mentions'])):
            dicionariobase = tweet ['entities']['user_mentions'][i].copy()
            dicionariobase.pop('indices', None)
            df = pd.DataFrame(dicionariobase, index= [0])
            df = df.rename(columns={
                'screen_name' : 'entities_screen_name',
                'name' : 'entities_name',
                'id' : 'entities_id',
                'id_str' : 'entities_id_str'
            })
            user_mentions.append(df)

            dfs = []
            for i in user_mentions:
                dfs.append(
                    pd.concat([df_tratado.copy(), i], axis=1)
                )
            df_final = pd.concat(dfs, ignore_index=True)
    except:
            return None
    return df_final

In [111]:
tweet_para_df(parsed_tweets[0])

In [112]:
%%time
## Vamos iterar e transformar todos os tweets
parseados = [tweet_para_df(tweet) for tweet in parsed_tweets]

Wall time: 3min 34s


In [118]:
# Eliminar as posições vazias da lista
parseados = [i for i in parseados if i is not None]

In [119]:
len(parseados)

15569

In [120]:
tratado = pd.concat(parseados, ignore_index=True)

In [121]:
tratado

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,user_verified,user_followers_count,user_friends_count,user_created_at,entities_screen_name,entities_name,entities_id,entities_id_str,possibly_sensitive,extended_entities
0,Mon Mar 08 12:55:32 +0000 2021,1368908241531514884,1368908241531514884,RT @WHCOS: Elections have consequences.,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,False,1979,4191,Sat Aug 26 14:36:10 +0000 2017,WHCOS,Ronald Klain,1351284403801108481,1351284403801108481,NaN,NaN
1,Mon Mar 08 12:55:32 +0000 2021,1368908241997037568,1368908241997037568,RT @SteveRattner: The top 20% of households re...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,False,7060,7060,Thu Nov 10 20:13:06 +0000 2016,SteveRattner,Steven Rattner,401625957,401625957,NaN,NaN
2,Mon Mar 08 12:55:32 +0000 2021,1368908242097700865,1368908242097700865,RT @MSNBC: Opinion | @FrankFigliuzzi1: The Sen...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,False,529,970,Sat Mar 14 04:36:08 +0000 2009,MSNBC,MSNBC,2836421,2836421,NaN,NaN
3,Mon Mar 08 12:55:32 +0000 2021,1368908242097700865,1368908242097700865,RT @MSNBC: Opinion | @FrankFigliuzzi1: The Sen...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,False,529,970,Sat Mar 14 04:36:08 +0000 2009,FrankFigliuzzi1,Frank Figliuzzi,929823128259387392,929823128259387392,NaN,NaN
4,Mon Mar 08 12:55:33 +0000 2021,1368908244366856193,1368908244366856193,RT @GeoffRBennett: NEW: Biden to order review ...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,False,2180,2967,Tue Mar 03 20:33:38 +0000 2009,GeoffRBennett,Geoff Bennett,19502995,19502995,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18848,Mon Mar 08 13:45:09 +0000 2021,1368920727605436417,1368920727605436417,RT @KateSullivanDC: Biden has nominated two fe...,"<a href=""http://twitter.com/download/iphone"" r...",False,None,None,None,None,...,False,13515,13943,Tue Mar 15 14:57:53 +0000 2016,KateSullivanDC,Kate Sullivan,241684668,241684668,NaN,NaN
18849,Mon Mar 08 13:45:09 +0000 2021,1368920727936712713,1368920727936712713,RT @CREWcrew: We can’t emphasize how outrageou...,"<a href=""http://twitter.com/download/android"" ...",False,None,None,None,None,...,False,278,309,Sat Oct 10 00:38:47 +0000 2020,CREWcrew,Citizens for Ethics,22187085,22187085,False,NaN
18850,Mon Mar 08 13:45:09 +0000 2021,1368920728121323524,1368920728121323524,RT @donwinslow: Please watch and share.\n\n@Re...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,None,None,None,None,...,False,4054,5001,Sun Dec 18 01:55:35 +0000 2016,donwinslow,Don Winslow,255812611,255812611,NaN,NaN
18851,Mon Mar 08 13:45:09 +0000 2021,1368920728121323524,1368920728121323524,RT @donwinslow: Please watch and share.\n\n@Re...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,None,None,None,None,...,False,4054,5001,Sun Dec 18 01:55:35 +0000 2016,ReverendWarnock,Senator Reverend Raphael Warnock,1221242033530195970,1221242033530195970,NaN,NaN
